# Apriori-Algorithmus
Implementieren Sie den Apriori-Algorithmus zum Finden häufiger Item Sets bei einer gegebenen Menge von Transaktionen. Nutzen Sie das gegebene Grundgerüst und beachten Sie folgende Anforderungen:
- Hinweis: Achten Sie besonders bei der Apriori Kandidatengenerierung darauf, dass Sie den Support möglichst effizient und nur von möglichst wenigen Itemsets berechnen.
- Die Ausgabe des Programms ist eine Liste aller Frequent Item Sets zusammen mit ihrem jeweiligen Support. Diese sollen in einer sinnvollen Datenstruktur zurückgegeben werden.
- Sie können Ihr Programm zum Beispiel auf dem Datensatz warenkorb.csv testen.
- Ihre Implementierung können Sie bis zum 09.01.2023 23:59 im Moodle abgeben.
- Sie dürfen die externen Pakete collecions und numpy verwenden. Aus dem Paket collections sollte vor allen Dingen die Klasse *Counter* für Sie hilfreich sein.

## Zusatzaufgabe für 6 ECTS:
**Hinweis**: Bearbeiten Sie diese Aufgaben nur, falls Sie am Praktikum für 6 ECTS-Punkte teilnehmen, jedoch nicht, falls Sie am Praktikum für 3 ECTS-Punkte teilnehmen.
### Anti-Monotone Constraints
- In der Vorlesung haben Sie anti-monotone Constraints zur Einschränkung der relevanten Itemsets kennengelernt.
- Implementieren Sie eine Möglichkeit, mittels des Parameters *prices* ein Dictionary mitzugeben, welches die Preise der einzelnen Items mitgibt.
- Durch den Parameter *max_price* soll dann eine mögliche Preisgrenze gegeben werden. Das heißt, es sollen nur frequent Itemsets zurückgegeben werden, welche nicht mehr als *max_price* kosten. Falls *max_price* den Wert None hat, soll der Constraint des maximalen Preises nicht genutzt werden.
- Nutzen Sie die Anti-Monotonie des Constraints aus, um die Generierung unnötiger Kandidaten zu vermeiden. Es ist NICHT ausreichend, alle frequent Itemsetzs erst zu berechnen um dann ganz zum Schluss die "zu teuren" zu entfernen!

### Konfidenz und Interessantheit
- Der Apriorie Algorithmus liefert Ihnen die Frequent Itemsets und ihren Support. Berechnen Sie aus diesen die Assoziationsregeln, welche eine vorgegebene Konfidenz überschreiten.
- Implementieren Sie die Funktion *interestingness* so, dass sie die Assozationsregeln, welche eine gewisse Konfidenz überschreiten und deren Interessantheit zurückgeben!

## Erinnerung
Bitte vergessen Sie nicht, sich offiziell im Praktikum einzuschreiben! Sollten Sie Fragen dazu haben, wenden Sie sich bitte per E-Mail an stubbemann@cs.uni-kassel.de.

# Daten laden
Diesen Teil bitte nicht verändern.

In [1]:
with open('warenkorb.csv') as f:
    data = []
    for line in f.readlines():
        items = [item.strip() for item in line.split(',')]
        items.sort()
        data.append(items)
data

[['Bier', 'Chips', 'Windeln'],
 ['Chips', 'TV-Zeitschrift'],
 ['Bier', 'Chips', 'TV-Zeitschrift'],
 ['Bier', 'Windeln', 'Zahnpasta'],
 ['Chips', 'Zahnpasta'],
 ['Bier', 'Chips', 'TV-Zeitschrift'],
 ['Bier', 'Windeln'],
 ['Chips', 'TV-Zeitschrift']]

In [2]:
prices = {"Chips": 2.5,
          "Bier": 5,
          "TV-Zeitschrift": 2,
          "Windeln": 1.5,
          "Zahnpasta": 2}

# Implementierung
Implementieren Sie in diesem Teil Ihre Lösung. Die gegebenen Datenstrukturen und Methoden können Sie als Hilfestellung (insbesondere für eine effiziente Lösung) benutzen, müssen Sie aber nicht. Nur die Apriori-Methode sollte so funktionieren wie  gegeben. Sie können natürlich weitere Methoden hinzufügen.

In [4]:
from collections import Counter
import numpy as np

def item_count(transactions):
    c = Counter([item for t in transactions for item in t])
    return c.most_common()

def apriori(transactions,
            prices=prices,
            max_price=None,
            minsup=0.01):
    # The parameters prices and max_price are just needed for 6ECTS!
    l = {}
    #1 Element Itemsets aus Transaktions
    l1 = item_count(transactions)
    dct = dict((Itemset(x),y/len(transactions)) for x, y in l1 if y/len(transactions) >= minsup)        
    l.update(dct)
    k = 2 
    currentList = [key for key in dct.keys()]
    #print(currentList)
    dct.clear()
    
    candidate = aprioriKandidatenGenerierung(currentList,minsup,k)
#     print(candidate)
    dct = returnItemsWithMinSupport(transactions,minsup,k,candidate)
#     print(dct)
     
#     while len(currentList)!= 0:
#         candidate = aprioriKandidatenGenerierung(currentList,minsup,k)
#         #für jede Transaktion Kandidate überprüfen und Zähler hochsetzen
#         dct = returnItemsWithMinSupport(transactions,minsup,k,candidate)
#         l.update(dct)
#         currentList.clear()
#         currentList = [key for key in dct.keys()]
#         k += 1
    
    return l

def aprioriKandidatenGenerierung(currentList,minsup,k):
    candidate = []
    if k == 2:
        lenSet = len(currentList)
        for i in range(lenSet):
            j = i + 1
            while j != lenSet:
                candidateTmp = Itemset(currentList[i].items + ' ' + currentList[j].items)
                candidate.append(candidateTmp)
                j +=1
    
    return candidate
            

def returnItemsWithMinSupport(transactions,minsup,k,candidate):
    # keys in dictionary basteln und values mit 0 initialisieren
    dct = {item: 0 for item in candidate}
    print(dct)
    #jede Element nach k basteln dann  dann Zähler hochzählen
    #kTmp = k - 1
        
    for transaction in transactions: 
#         print(transaction)
        lenTransaction = len(transaction)
        #Element zu vergleichen basteln mit k Länge
        tmpCompareTransactionList = []
        for i in range(lenTransaction):
            j = i + 1
            while j != lenTransaction:
                tmpCompare = transaction[i] + ' ' + transaction[j]
#                 print(tmpCompare)
                j +=1
                tmpCompareTransactionList.append(tmpCompare)
#         print(tmpCompareTransactionList)
        #mit dictionaryskey vergleichen
        for item in tmpCompareTransactionList:
            if dct[Itemset(tmpCompare)] != None:
                dct[Itemset(tmpCompare)] += 1 
        print(dct)
    
    return dct
        
def interestingness(l, min_conf):
    """
    This function is only needed for 6ECTS!
    The list l contains all frequent itemsets and their support.
    Compute fom all frequent Itemsets X and all their subsets A the interestingness of the rule
    A -> X\A if this rule has at least the confidence min_conf.
    Returns a list of tuples where each tuple has the form (A, X\A , interstingness of A -> X\A)
    """
    # TODO
    pass


class Itemset():
    """ A hashable itemset that can be used as a key in dictionaries or as an element in sets.
    You can change this class how ever you like. """
    
    def __init__(self, items):
        self.items = items
    
    def __str__(self):
        return str(self.items)
    
    def __repr__(self):
        return str(self)
    
    def __eq__(self, other):
        return len(self.items) == len(other.items) and set(self.items).issubset(other.items)
    
    def __hash__(self):
        return hash(str(self))
    
#     def __add__(self, other):
#         newItem = self.items + other.items
#         return self(newItem)
    
    
# Calculate frequent itemsets for the given data-set and print the result.
l = apriori(data, minsup=0.1)
l

{Chips Bier: 0, Chips TV-Zeitschrift: 0, Chips Windeln: 0, Chips Zahnpasta: 0, Bier TV-Zeitschrift: 0, Bier Windeln: 0, Bier Zahnpasta: 0, TV-Zeitschrift Windeln: 0, TV-Zeitschrift Zahnpasta: 0, Windeln Zahnpasta: 0}
{Chips Bier: 0, Chips TV-Zeitschrift: 0, Chips Windeln: 3, Chips Zahnpasta: 0, Bier TV-Zeitschrift: 0, Bier Windeln: 0, Bier Zahnpasta: 0, TV-Zeitschrift Windeln: 0, TV-Zeitschrift Zahnpasta: 0, Windeln Zahnpasta: 0}
{Chips Bier: 0, Chips TV-Zeitschrift: 1, Chips Windeln: 3, Chips Zahnpasta: 0, Bier TV-Zeitschrift: 0, Bier Windeln: 0, Bier Zahnpasta: 0, TV-Zeitschrift Windeln: 0, TV-Zeitschrift Zahnpasta: 0, Windeln Zahnpasta: 0}
{Chips Bier: 0, Chips TV-Zeitschrift: 4, Chips Windeln: 3, Chips Zahnpasta: 0, Bier TV-Zeitschrift: 0, Bier Windeln: 0, Bier Zahnpasta: 0, TV-Zeitschrift Windeln: 0, TV-Zeitschrift Zahnpasta: 0, Windeln Zahnpasta: 0}
{Chips Bier: 0, Chips TV-Zeitschrift: 4, Chips Windeln: 3, Chips Zahnpasta: 0, Bier TV-Zeitschrift: 0, Bier Windeln: 0, Bier Zahnpas

{Chips: 0.75,
 Bier: 0.625,
 TV-Zeitschrift: 0.5,
 Windeln: 0.375,
 Zahnpasta: 0.25}

## Tests für die 6ECTS Aufgaben
Löschen Sie die folgenden Zellen, wenn Sie nur 3ECTS benötigen!

In [ ]:
l = apriori(data, minsup=0.2, max_price=7)
l

In [ ]:
l = apriori(data, minsup=0.01)
i = interestingness(l, min_conf=0.3)
i

In [5]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))